# 双层网络，最基本网络结构，Cross-Entropy，ReLU，Init


# 导入库

In [1]:
import numpy as np
from scipy import ndimage
import matplotlib.pyplot as plt
from load_data import *
%matplotlib inline

# 载入数据

In [2]:
training_data, validation_data, test_data = load_data_wrapper()

# 训练参数

In [3]:
n_epoch = 30
learning_rate = 0.1
batch_size = 10

# 网络结构

In [4]:
n_node_input = 784
n_node_hidden = 100
n_node_output = 10

# 权重与偏置

In [5]:
W_init = 'xavier'
b_init = 'zero'

if W_init == 'normal':
    W2 = np.random.randn(n_node_hidden, n_node_input)
    W3 = np.random.randn(n_node_output, n_node_hidden)
elif W_init == 'xavier':
    W2 = np.random.randn(n_node_hidden, n_node_input) / np.sqrt(n_node_input)
    W3 = np.random.randn(n_node_output, n_node_hidden) / np.sqrt(n_node_hidden)
else : # 'he'
    W2 = np.random.randn(n_node_hidden, n_node_input) / np.sqrt(n_node_input/2)
    W3 = np.random.randn(n_node_output, n_node_hidden) / np.sqrt(n_node_hidden/2)

if b_init == 'normal':
    b2 = np.random.randn(n_node_hidden, 1)
    b3 = np.random.randn(n_node_output, 1)
else: # 'zero'
    b2 = np.zeros([n_node_hidden, 1])
    b3 = np.zeros([n_node_output, 1])

# 激活函数

In [6]:
def sigmoid(z):
    """The sigmoid function."""
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    """Derivative of the sigmoid function."""
    return sigmoid(z)*(1-sigmoid(z))

def relu(z):
    """The relu function."""
    return np.maximum(z, 0)

def relu_prime(z):
    """Derivative of the relu function."""
    return np.where(z > 0, 1.0, 0.0)

# 开始训练

In [7]:
### Training
test_errors = []
training_errors = []
n = len(training_data)


for j in range(n_epoch):

    ## Stochastic Gradient Descent
    np.random.shuffle(training_data)

    # for each batch
    sum_of_training_error = 0
    for k in range(0, n, batch_size):
        batch = training_data[k:k+batch_size]

        # average gradient for samples in a batch
        sum_gradient_b3 = 0
        sum_gradient_b2 = 0
        sum_gradient_W3 = 0
        sum_gradient_W2 = 0

        # for each sample
        for x, y in batch:
            ## Feed forward

            a1 = x
            z2 = np.dot(W2, a1) + b2
            a2 = relu(z2)
            z3 = np.dot(W3, a2) + b3
            a3 = sigmoid(z3)

            ## Backpropagation

            # Step 1: Error at the output layer [Cross-Entropy Cost]
            delta_3 = (a3-y)
            # Step 2: Error relationship between two adjacent layers
            delta_2 =  relu_prime(z2)*np.dot(W3.transpose(), delta_3)
            # Step 3: Gradient of C in terms of bias
            gradient_b3 = delta_3
            gradient_b2 = delta_2
            # Step 4: Gradient of C in terms of weight
            gradient_W3 = np.dot(delta_3, a2.transpose())
            gradient_W2 = np.dot(delta_2, a1.transpose())

            # update gradients
            sum_gradient_b3 += gradient_b3
            sum_gradient_b2 += gradient_b2
            sum_gradient_W3 += gradient_W3
            sum_gradient_W2 += gradient_W2

            ## Training Error
            sum_of_training_error += int(np.argmax(a3) != np.argmax(y))

        # Update Biases
        b3 -= learning_rate * sum_gradient_b3 / batch_size
        b2 -= learning_rate * sum_gradient_b2 / batch_size

        # Update Weights
        W3 -= learning_rate * sum_gradient_W3 / batch_size
        W2 -= learning_rate * sum_gradient_W2 / batch_size

    # Report Training Error
    print("[TRAIN_ERROR] Epoch %02d: %5d / %05d" % (j, sum_of_training_error, n))
    training_errors.append(np.float(sum_of_training_error) / n)

    ### Test
    n_test = len(test_data)
    sum_of_test_error = 0
    for x, y in test_data:
        ## Feed forward

        a1 = x
        z2 = np.dot(W2, a1) + b2
        a2 = relu(z2)
        z3 = np.dot(W3, a2) + b3
        a3 = sigmoid(z3)

        ## Test Error
        # in test data, label info is a number not one-hot vector as in training data
        sum_of_test_error += int(np.argmax(a3) != y)

    # Report Test Error
    print("[ TEST_ERROR] Epoch %02d: %5d / %05d" % (j, sum_of_test_error, n_test))

    test_errors.append(np.float(sum_of_test_error)/n_test)
print("done!")

[TRAIN_ERROR] Epoch 00:  3685 / 50000
[ TEST_ERROR] Epoch 00:   397 / 10000
[TRAIN_ERROR] Epoch 01:  1712 / 50000
[ TEST_ERROR] Epoch 01:   292 / 10000
[TRAIN_ERROR] Epoch 02:  1230 / 50000
[ TEST_ERROR] Epoch 02:   255 / 10000
[TRAIN_ERROR] Epoch 03:   986 / 50000
[ TEST_ERROR] Epoch 03:   265 / 10000
[TRAIN_ERROR] Epoch 04:   802 / 50000
[ TEST_ERROR] Epoch 04:   237 / 10000
[TRAIN_ERROR] Epoch 05:   650 / 50000
[ TEST_ERROR] Epoch 05:   211 / 10000
[TRAIN_ERROR] Epoch 06:   553 / 50000
[ TEST_ERROR] Epoch 06:   226 / 10000
[TRAIN_ERROR] Epoch 07:   454 / 50000
[ TEST_ERROR] Epoch 07:   224 / 10000
[TRAIN_ERROR] Epoch 08:   378 / 50000
[ TEST_ERROR] Epoch 08:   223 / 10000
[TRAIN_ERROR] Epoch 09:   363 / 50000
[ TEST_ERROR] Epoch 09:   226 / 10000
[TRAIN_ERROR] Epoch 10:   271 / 50000
[ TEST_ERROR] Epoch 10:   206 / 10000
[TRAIN_ERROR] Epoch 11:   226 / 50000
[ TEST_ERROR] Epoch 11:   218 / 10000
[TRAIN_ERROR] Epoch 12:   207 / 50000
[ TEST_ERROR] Epoch 12:   218 / 10000
[TRAIN_ERROR